# Weather data by city, year

MakeAPICall code citation: https://www.educative.io/answers/how-to-make-api-calls-in-python <br>
Lat/Long dataset: https://www.kaggle.com/datasets/camnugent/california-housing-feature-engineering?resource=download <br>
Weather API: https://open-meteo.com/en/docs/historical-weather-api#latitude=37.34&longitude=-121.89&start_date=2016-01-01&end_date=2017-01-01&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&timeformat=unixtime

In [34]:
import numpy as np

In [51]:
import pandas as pd

df = pd.read_csv(r'.\Data\cal_cities_lat_long.csv')
print(df)
#print(df[df['Name'] == 'Los Angeles'])
coordinates = df.head(2)
coordinates

             Name   Latitude   Longitude
0        Adelanto  34.582769 -117.409214
1    Agoura Hills  34.153339 -118.761675
2         Alameda  37.765206 -122.241636
3          Albany  37.886869 -122.297747
4        Alhambra  34.095286 -118.127014
..            ...        ...         ...
454      Woodland  38.678517 -121.773297
455   Yorba Linda  33.888625 -117.813111
456         Yreka  41.735419 -122.634472
457     Yuba City  39.140447 -121.616911
458       Yucaipa  34.033625 -117.043086

[459 rows x 3 columns]


,Name,Latitude,Longitude
0,Adelanto,34.582769,-117.409214
1,Agoura Hills,34.153339,-118.761675


In [105]:
import requests
import json
from scipy import stats as st

def MakeApiCall(api, coordinates):
    weather = pd.DataFrame(columns = ['city','year','weathercode','temperature_2m_max','temperature_2m_min','temperature_2m_mean','precipitation_sum','rain_sum','snowfall_sum','precipitation_hours','windspeed_10m_max','windgusts_10m_max'])
    for i in range(len(coordinates)): 
        parameters = {
            "latitude": coordinates['Latitude'][i],
            "longitude": coordinates['Longitude'][i],
            "start_date": '2014-01-01',
            "end_date": '2018-12-31',
            "timezone": 'America/Los_Angeles',
            "daily": ['weathercode','temperature_2m_max','temperature_2m_min','temperature_2m_mean','precipitation_sum','rain_sum','snowfall_sum','precipitation_hours','windspeed_10m_max','windgusts_10m_max'],
            'temperature_unit': 'fahrenheit',
            'windspeed_unit': 'mph',
            'timeformat': 'unixtime'
        }
        city = coordinates['Name'][i]
        weather = get_user_data(api, parameters, city, weather)
    return weather
    
def get_user_data(api, parameters, city, dataset):
    response = requests.get(f"{api}", params=parameters, verify = False)
    if response.status_code == 200:
        print("Sucessfully fetched the data with parameters provided")
        #formatted_print(response.json())
        return data(response.json(), city, dataset)
    else:
        print(
            f"Hello person, there's a {response.status_code} error with your request")

def formatted_print(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

def data(obj, city, dataset):
    metrics = ['weathercode','temperature_2m_max','temperature_2m_min','temperature_2m_mean','precipitation_sum','rain_sum','snowfall_sum','precipitation_hours','windspeed_10m_max','windgusts_10m_max']
    app = {'city':city}
    dates = [[0,364], [365,729], [730,1095], [1096,1460], [1461,1825]]
    year = [2014, 2015, 2016, 2017, 2018]
    count = 0
    for date in dates:
        for metric in metrics:
            if metric == 'weathercode':
                value = st.mode(obj['daily'][metric][date[0]: date[1]])[0][0]
            elif metric in ('temperature_2m_max','temperature_2m_min','temperature_2m_mean','windspeed_10m_max','windgusts_10m_max'):
                value = np.average(obj['daily'][metric][date[0]: date[1]])
            elif metric in ('precipitation_sum', 'rain_sum','snowfall_sum','precipitation_hours'):
                value = np.sum(obj['daily'][metric][date[0]: date[1]])
            else:
                return 'Error'
            app[metric] = value
        app['year'] = year[count]
        count += 1
        dataset = dataset.append(app, ignore_index = True)
    return dataset

import warnings

warnings.filterwarnings("ignore")
    
api_call = MakeApiCall("https://archive-api.open-meteo.com/v1/archive?", df)

api_call.to_csv(r'.\Data\Weather.csv', index=False)

Sucessfully fetched the data with parameters provided
Sucessfully fetched the data with parameters provided
